# EV Charging – OpenChargeMap API extract & postcode cleaning

This notebook calls the OpenChargeMap (OCM) API to download all public charge point locations for Great Britain, extracts two flat tables:

- `ocm_sites.csv` – one row per charging site
- `ocm_plugs.csv` – one row per plug/connection

It also prepares a `postcode_la.csv` lookup from the ONS Postcode Directory (ONSPD), which maps postcodes to 2025 local authority codes (`lad25cd`).

These CSVs are the raw inputs for the `ev_charging` Postgres schema.


## Contents

1. Setup and configuration  
2. Call OpenChargeMap API  
3. Build `sites` table and basic postcode cleaning  
4. Build `plugs` table  
5. Create postcode → LAD lookup from ONSPD  
6. Summary and next steps


## 1. Setup and configuration

Import Python libraries, define the OCM API endpoint and parameters, and load the API key from a local secrets file


In [1]:
import requests
import pprint
import pandas as pd

%run "secrets/api_keys.ipynb"

OCM_URL = "https://api.openchargemap.io/v3/poi"

OCM_PARAMS = {
    "countrycode": "GB",
    "maxresults": 100000,
    "output": "json"
}

OCM_HEADERS = {"X-API-Key": OCM_API_KEY}


## 2. Call OpenChargeMap API

Here I call the OCM `/v3/poi` endpoint for all GB charge points, using a high `maxresults` value to retrieve all records. I then inspect the first JSON object to understand the structure.
After this step I work from the CSV outputs to avoid repeated calls to the public API.


In [2]:
response = requests.get(OCM_URL, params=OCM_PARAMS, headers=OCM_HEADERS)
response.raise_for_status()

data = response.json()
print("Number of sites returned:", len(data))

pprint.pprint(data[0])


Number of sites returned: 27456
{'AddressInfo': {'AccessComments': 'Luton ',
                 'AddressLine1': '300',
                 'AddressLine2': None,
                 'ContactEmail': 'Tettehdaniel1981@gmail.com',
                 'ContactTelephone1': '07380585602',
                 'ContactTelephone2': '07501061728',
                 'Country': {'ContinentCode': 'EU',
                             'ID': 1,
                             'ISOCode': 'GB',
                             'Title': 'United Kingdom'},
                 'CountryID': 1,
                 'Distance': None,
                 'DistanceUnit': 0,
                 'ID': 469730,
                 'Latitude': -0.0005212262455955852,
                 'Longitude': -0.00015884563913459715,
                 'Postcode': 'LU2 0EU ',
                 'RelatedURL': None,
                 'StateOrProvince': 'Bedfordshire ',
                 'Title': 'Hitchin Road ',
                 'Town': 'Luton '},
 'Connections': [],
 'DataPro

## 3. Build `sites` table and basic postcode cleaning

This section flattens the OCM JSON into a site-level table with one row per charging site.  
Key fields: site ID, title, postcode, latitude/longitude, operator, usage type, and reported number of points.

I also add a basic `postcode_clean` field by uppercasing and removing non-alphanumeric characters, which will later be matched to the ONS Postcode Directory.


In [3]:
site_rows = []

for p in data:
    a = p.get("AddressInfo") or {}

    site_rows.append({
        "site_id": p.get("ID"),
        "site_title": a.get("Title"),
        "postcode": a.get("Postcode"),
        "lat": a.get("Latitude"),
        "lon": a.get("Longitude"),
        "operator": (p.get("OperatorInfo") or {}).get("Title"),
        "usage_type": (p.get("UsageType") or {}).get("Title"),
        "reported_points": p.get("NumberOfPoints"),
    })

sites = pd.DataFrame(site_rows)
print("Sites rows:", len(sites))
sites.head()


Sites rows: 27456


,site_id,site_title,postcode,lat,lon,operator,usage_type,reported_points
0,469341,Hitchin Road,LU2 0EU,-0.000521,-0.000159,(Unknown Operator),"Private - For Staff, Visitors or Customers",1.0
1,469284,Zapgo - The Stanley Club,L40 5TN,53.600397,-2.845272,(Unknown Operator),Public,4.0
2,464404,Tim Hortons,ML1 2AF,55.781336,-3.978248,E.ON Drive,Public,2.0
3,462390,Gawcott Fields Business Park,MK18 1TN,51.462419,-0.157305,Monta,Public - Membership Required,2.0
4,462349,2 Wanless Road,None,51.464518,-0.101029,Clenergy,Public,1.0


In [4]:
sites["postcode_clean"] = (
    sites["postcode"]
        .str.upper()
        .str.replace(r"[^A-Z0-9]", "", regex=True)
        .replace("", pd.NA)  # empty strings -> NA
)

print("Duplicate site_id rows:", sites["site_id"].duplicated(keep=False).sum())
print("Missing raw postcode:", sites["postcode"].isna().sum())
print("Missing postcode_clean:", sites["postcode_clean"].isna().sum())

sites.head()

Duplicate site_id rows: 0
Missing raw postcode: 58
Missing postcode_clean: 83


,site_id,site_title,postcode,lat,lon,operator,usage_type,reported_points,postcode_clean
0,469341,Hitchin Road,LU2 0EU,-0.000521,-0.000159,(Unknown Operator),"Private - For Staff, Visitors or Customers",1.0,LU20EU
1,469284,Zapgo - The Stanley Club,L40 5TN,53.600397,-2.845272,(Unknown Operator),Public,4.0,L405TN
2,464404,Tim Hortons,ML1 2AF,55.781336,-3.978248,E.ON Drive,Public,2.0,ML12AF
3,462390,Gawcott Fields Business Park,MK18 1TN,51.462419,-0.157305,Monta,Public - Membership Required,2.0,MK181TN
4,462349,2 Wanless Road,None,51.464518,-0.101029,Clenergy,Public,1.0,None


In [5]:
sites.to_csv("ocm_sites.csv", index=False)

## 4. Build `plugs` table

This section explodes the nested `Connections` array into a plug-level table, with one row per connection at each site.

Key fields: site ID, connection ID, connection/plug type, power in kW, quantity, and status.


In [6]:
plug_rows = []

for p in data:
    for c in (p.get("Connections") or []):
        plug_rows.append({
            "site_id": p.get("ID"),
            "connection_id": c.get("ID"),
            "connection_type": (c.get("ConnectionType") or {}).get("Title"),
            "level": (c.get("Level") or {}).get("Title"),
            "current_type": (c.get("CurrentType") or {}).get("Title"),
            "power_kw": c.get("PowerKW"),
            "quantity": c.get("Quantity"),
            "status": (c.get("StatusType") or {}).get("Title"),
        })

plugs = pd.DataFrame(plug_rows)
print("Plugs rows:", len(plugs))
plugs.head()

Plugs rows: 47204


,site_id,connection_id,connection_type,level,current_type,power_kw,quantity,status
0,469284,787696,CCS (Type 2),Level 3: High (Over 40kW),DC,200.0,4.0,Operational
1,464404,781941,CCS (Type 2),Level 3: High (Over 40kW),DC,200.0,NaN,Operational
2,462390,777729,Type 2 (Socket Only),Level 2 : Medium (Over 2kW),None,7.2,2.0,Operational
3,462349,777663,Type 2 (Socket Only),Level 2 : Medium (Over 2kW),AC (Three-Phase),22.0,1.0,Operational
4,462349,777664,Type 2 (Socket Only),Level 2 : Medium (Over 2kW),AC (Three-Phase),22.0,1.0,Operational


In [7]:
plugs.to_csv("ocm_plugs.csv", index=False)

## 5. Create postcode to LAD lookup from ONSPD

To link OCM sites to local authorities, I use the ONS Postcode Directory (ONSPD).  
Here I:

- Read the raw ONSPD CSV (Aug 2025 extract),
- Apply the same postcode cleaning logic as for OCM,
- Keep only the cleaned postcode and 2025 LAD code (`lad25cd`),
- Drop duplicates to create a compact postcode to LAD lookup table.


In [9]:
ons = pd.read_csv("../Data/raw data/ONSPD_AUG_2025_UK.csv", dtype=str, low_memory=False)

ons["pcds_clean"] = (
    ons["pcds"]
        .str.upper()
        .str.replace(r"[^A-Z0-9]", "", regex=True)
)

keep_cols = ["pcds_clean", "lad25cd"]

postcode_la = (
    ons[keep_cols]
    .dropna(subset=["pcds_clean", "lad25cd"])
    .drop_duplicates()
)

print("Postcode-LAD rows:", len(postcode_la))
postcode_la.head()

Postcode-LAD rows: 2706777


,pcds_clean,lad25cd
0,AB10AA,S12000033
1,AB10AB,S12000033
2,AB10AD,S12000033
3,AB10AE,S12000034
4,AB10AF,S12000033


In [ ]:
postcode_la.to_csv("postcode_la.csv", index=False)

## 6. Summary and next steps

In this notebook I:

- Called the OpenChargeMap API for all GB charge points and inspected the JSON structure.
- Built a site-level table (`ocm_sites.csv`) with basic postcode cleaning and QA on missing/duplicate IDs.
- Built a plug-level table (`ocm_plugs.csv`) with one row per connection.
- Created a compact postcode → LAD lookup (`postcode_la.csv`) from the ONS Postcode Directory using consistent postcode cleaning.

These CSVs are loaded into Postgres as the `sites`, `plugs` and `postcode_la` tables in the `ev_charging` schema, where I then define analytical views such as `la_supply_kpis` for the Tableau analysis.
